In [6]:
from POSEIDON.utility import read_high_res_data
from POSEIDON.high_res import fit_uncertainties
import numpy as np
import matplotlib.pyplot as plt
import pickle

data_dir = "./data/WASP-76b"
data = read_high_res_data(data_dir, method="sysrem", spectrum_type="transmission")
data_raw = data["data_raw"]
data_raw[data_raw < 0] = 0
Ndet, Nphi, Npix = data_raw.shape
wl_grid = data["wl_grid"]

In [ ]:
plt.figure(figsize=(17, 5))
plt.imshow(data_raw[0], aspect="auto")

In [ ]:
import pickle

output_path = "./CC_output/WASP-76b/"  # Could modify output path here.
# uncertainties = fit_uncertainties(data_raw, NPC=5)
uncertainties = pickle.load(open(data_dir + "/uncertainties.pic", "rb"))
data = read_high_res_data(data_dir, method="sysrem", spectrum_type="transmission")[
    "data_raw"
]

In [ ]:
from POSEIDON.high_res import fast_filter

for i in range(len(data)):
    order = data[i]

    median = np.median(order, axis=0)
    median[median == 0] = np.mean(median)
    order_norm = order / median

    uncertainty = uncertainties[i]

    uncertainty_norm = uncertainty / median

    uncertainties[i] = uncertainty_norm
    data[i] = order_norm

residuals, Us = fast_filter(data, uncertainties, Niter=15)

In [ ]:
plt.figure(figsize=(17, 5))
plt.imshow(residuals[2], aspect="auto", origin="lower", cmap="gray")
plt.savefig(output_path + "filtered.pdf", fmt="pdf")

In [ ]:
Bs = np.zeros((Ndet, Nphi, Nphi))

for j in range(Ndet):
    U = Us[j]
    L = np.diag(1 / np.mean(uncertainties[j], axis=-1))
    B = U @ np.linalg.pinv(L @ U) @ L
    Bs[j] = B

In [ ]:
plt.figure(figsize=(17, 5))
plt.plot(wl_grid[10], residuals[10][60])
plt.ticklabel_format(useOffset=False)

In [ ]:
pickle.dump([Bs, residuals], open(data_dir + "/fast_filter.pic", "wb"))